## Modeling 
The goal of this notebook is to confirm what model will be used for the offensive coordinator and ensure it can be effectively implemented with the data from 01_exploration before preprocessing steps are ported to C++. Eventually the modeling steps in this notebook will be done in a train.py file.   

**Tasks:**  
1. Load Cleaned Data
2. Model Data
3. Evaluate Model

**1. Load Cleaned Data**